### Hellman-Feynman Analysis Figure
This notebook makes the 4 panel hellman-feynman analysis figure.

In [ ]:
# configure plotting parameters

%config InlineBackend.figure_format='svg'
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import font_manager
plt.rcParams['font.family'] = 'Helvetica'
plt.rcParams['font.size'] = 14

### Define general use plotting functions

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# general plotting function - default to normalize
def plot_data(dataframe, axis,  y, ylabel, x = "bond_angle[deg]", normalize = True, 
              legend = True, legend_loc = "upper left", markersize = 3, linewidth = 1):

    # group the dataframe by heteroatom and plot the results
    colors = iter(["dodgerblue", "dimgray"])
    linestyles = iter(["solid", "dashed"])
    markerstyles = iter(["o", "D"])
    for series_name, group in dataframe.groupby('heteroatom'):

        group.sort_values(x, inplace=True)
        x_data = group[x]
        y_data = group[y]

        if normalize == True:
            y_data = normalize_data(y_data)

        mycolor = next(colors)
        axis.plot(x_data, y_data, markersize = markersize, mfc=mycolor, mec=mycolor, linewidth = linewidth,  marker=next(markerstyles), 
                  linestyle=next(linestyles), label=f"C{series_name}$_3$ radical", color="black")

    axis.set_xlabel('\u03B8\u00B0'), axis.set_ylabel(ylabel)

    ymin, ymax = axis.get_ylim()
    vertical_lines = [111.4, 120]

    axis.vlines(vertical_lines, ymin, ymax, color="red")

    if legend == True:
        axis.legend(loc = legend_loc, facecolor="white", framealpha=1)


def normalize_data(data):
    # normalize to (0,1)
    normalized_data = (data - np.min(data)) / (np.max(data) - np.min(data))
    return normalized_data

### Make the figure

In [ ]:
import matplotlib.image as im
from matplotlib.offsetbox import (OffsetImage, AnnotationBbox)
import sys; sys.path.insert(0, './')

hellman_feynman_dataframe = pd.read_csv("./merged_data.csv")

fig2, axs2 = plt.subplots(2,2, figsize=(10,10))


y_data_list = ["C_tot_z_force[hartree/bohr]", "C_elec_z_force[hartree/bohr]", "C_nuc_z_force[hartree/bohr]"]
ylabel_list = ["total force (z) on C [hartree/bohr]", "electronic force (z) on C [hartree/bohr]", "nuclear force (z) on C [hartree/bohr]"]
normalize_boolean = [False, False, False]
panel_labels = list("ABC")
plot_indices = [(0,0), (0,1), (1,0)]
legend_loc = ["upper left", "upper left", "upper left"]
for i, id in enumerate(plot_indices):
    try:
        dataframe = hellman_feynman_dataframe
        
        plot_data(dataframe, axs2[id], y=y_data_list[i], ylabel=ylabel_list[i], normalize=normalize_boolean[i], legend_loc=legend_loc[i])
        axs2[id].text(-0.1, 1.1, panel_labels[i], transform=axs2[id].transAxes,
                   fontsize=20, fontweight='bold', va='top', ha='right')
        xmin, xmax = axs2[id].get_xlim()
        axs2[id].hlines(0, xmin, xmax, color="k", linewidth=1)
        
    except Exception as e:
        print(e)
        pass

# adding orbital images
origin_key = im.imread("origin_xyz_key.png")
image_pos = [(0,0), (0,0), (0,0)]

imagebox = OffsetImage(origin_key, zoom=0.3)
ab = AnnotationBbox(imagebox, (0.5,0.5), frameon=False)
axs2[1,1].add_artist(ab)


axs2[1,1].get_xaxis().set_visible(False)
axs2[1,1].get_yaxis().set_visible(False)
axs2[1,1].text(-0.1, 1.1, "D", transform=axs2[1,1].transAxes,
                   fontsize=20, fontweight='bold', va='top', ha='right')

axs2[0,0].set_xticks(np.arange(95,125,5))
axs2[1,0].set_xticks(np.arange(95,125,5))
axs2[0,1].set_xticks(np.arange(95,125,5))

fig2.tight_layout()